<h1><center> <span style="color:#0ad0f7">Hotel Booking Demand</span></center></h1>

## `Business Problem`

**Context**:
<br>
Terdapat sebuah perusahaan yang bergerak dibidang hospitality industry-khususnya perhotelan-dan berlokasi di Portugal, memiliki dataset yang telah diekstrak dari database hotel (Property Management System (PMS)). Hospitality industry lebih fokus dengan kepuasan pelanggan. Berdasarkan [The Big Four](https://www.hospitalitynet.org/opinion/4082318.html), ada bermacam-macam hospitality industry, seperti Food and Beverages, Travel and Tourism, Lodging, dan Recreation. Industri hotel termasuk ke dalam lodging, salah satu permasalahan yang sering terjadi pada industri hotel adalah pembatalan pesanan oleh pelanggan. Salah satu cara pemesanan kamar pada perusahaan hotel ini adalah dengan melakukan pemesanan terlebih dahulu sebelum tanggal menginap (booking system).

Target (is_canceled):

1 : pesanan yang dibatalkan (positif)

0 : pesanan yang **tidak** dibatalkan (negatif)

**Project Stakeholders**:
<br>
Stakeholders yang akan menggunakan model ini adalah:
1. Marketing & Sales Departement : Departement ini bertanggungjawab untuk memaksimalkan revenue dengan cara melakukan beberapa strategi marketing (ex: promosi/diskon) kepada pelanggan, sehingga dengan model ini Marketing dan Sales Departement bisa mengetahui pelanggan yang **tepat** untuk diberi promosi/diskon.
2. Finance Departement : Salah satu tugas Finance Departement adalah mengatur pemasukan dan pengeluaran keuangan perusahaan yang salah satunya diakibatkan dari promosi/campaign yang diadakan Marketing dan Sales Departement. 

Pada dasarnya, kedua departement ini saling berkesinambungan dan memiliki pertanyaan/tujuan yang sama, yaitu **bagaimana caranya memaksimalkan revenue perusahaan.**

**Prolem Statement**:
<br>
Ada beberapa problem yang sering terjadi di hotel industry, salah satunya adalah pembatalan pesanan yang telah dilakukan. Berdasarkan referensi dibawah, cancellation rate secara global berada pada angka **33%** pada tahun 2014 dan **40%** pada tahun 2018, sedangkan pada dataset ini cancellation rate-nya sebesar **37%**. Hal ini menunjukkan adanya peningkatan cancellation rate pada tahun-tahun tersebut dan tentu saja memberikan dampak **negatif** bagi revenue perusahaan. Revenue loss akibat dari pesanan yang dibatalkan dapat mencapai **15%**.

Reference:
1. [Global Cancellation Rate Hotel Reservations Reacehs 40 average](https://hospitalitytech.com/global-cancellation-rate-hotel-reservations-reaches-40-average)
2. [How Hotels Can Combat Rising Cancellation Rates](https://resources.emerchantpay.com/how-hotels-can-combat-rising-cancellation-rates)
3. [Revenue Loss Due to Cancellations](https://revenue-hub.com/three-most-common-trends-impacting-cancellation-rates/)

**Goals**:
<br>
Perusahaan ingin mengetahui atau **memprediksi apakah suatu pelanggan akan membatalkan pesanan atau tidak**. Dengan mengidentifikasi **pesanan mana yang kemungkinan dibatalkan**, pihak hotel dapat **menawarkan layanan lainnya** seperti, diskon, tiket masuk ke pertunjukkan/taman hiburan, ataupun hal lainnya. Akan tetapi, penawaran-penawaran ini mungkin tidak dapat selalu diterapkan, mengingat tingkat keekonomian/kesensitifan pelanggan berbeda-beda, contoh tamu perusahaan (coorporate guest). Penawaran ini dilakukan dengan tujuan pihak hotel sebisa mungkin perusahaan tidak kehilangan potential customer/pelanggan, sehingga dapat meningkatkan revenue perusahaan.

**Analytic Approach**:
<br>
Jadi yang akan kita lakukan adalah menganalisis data untuk menemukan pola yang membedakan pelanggan yang akan membatalkan pesanan atau tidak.

Kemudian kita akan membangun model `klasifikasi` yang akan membantu perusahaan untuk dapat memprediksi pelanggan yang akan membatalkan pesanan atau tidak.

**Metric Evaluation**:
<br>
Type I Error: False Positive
<br>
Kondisi di mana pelanggan terprediksi **membatalkan** pesanan padahal kenyataannya tidak membatalkan pesanan
<br>
`Konsekuensinya:` promosi tidak tepat sasaran, sehingga revenue tidak maksimal.

Type II Error: False Negative
<br>
Kondisi di mana pelanggan terprediksi tidak membatalkan pesanan padahal kenyataannya membatalkan pesanan
<br>
`Konsekuensinya:` pihak hotel kehilangan potential pelanggan dan tentu saja berdampak pada kehilangan revenue.

Berdasarkan referensi di bawah, pihak hotel mendapatkan **USD 16** dari setiap promosi yang diadakan. Oleh karena itu, berdasarkan hal-hal diatas, maka sebisa mungkin yang akan kita lakukan adalah membuat model yang dapat memprediksi **sebanyak mungkin kelas positif dan kelas negatif yang benar dan meminimalkan type I error dan type II error**. Jadi, nanti metric utama yang digunakan adalah `MCC (Matthew Correlation Coefficient)`.

Referensi:
1. [Hotel Marketing Budgets: How Much Is Actually Enough?](https://www.hospitalitynet.org/opinion/4090018.html)
2. [The Digital Marketing Experts for Independent Hotels & Management Companies](https://www.orourkehospitality.com/resources/topic/hospitality-digital-marketing/4-tips-for-planning-your-2022-hospitality-marketing-budget/)

ADD NEW METRIC (WARNING):
1. `Balanced Accuracy`
1. `MCC (Matthew coef correlation)`

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r"C:\Users\faisa\Desktop\DataSciencePurwadhika\Modul3ML\HotelBookingDemand\data\raw\data_hotel_booking_demand.csv")
data.shape

(83573, 11)

In [6]:
from collections import Counter

# Counter(data.is_canceled)

(data.is_canceled.value_counts(normalize=True) * 100).to_frame()

,is_canceled
0,63.172316
1,36.827684


Insight:
- Cancellation rate pada dataset ini sebesar 36% atau bisa dikatakan bahwa **4 dari 10 pesanan dibatalkan**. Berdasarkan jurnal pada problem statement, hal ini berpengaruh negatif terhadap revenue hotel.


In [11]:
from sklearn.model_selection import train_test_split

# label
y = data['is_canceled']

# split train dan test
dfTrain, dfTest = train_test_split(data, test_size=0.25, random_state=22, stratify=y)

# data train
X_train = dfTrain.drop(columns='is_canceled')
y_train = dfTrain['is_canceled']

# data test
X_test = dfTest.drop(columns='is_canceled')
y_test = dfTest['is_canceled']

display(dfTrain.shape, dfTest.shape)

(62679, 11)

(20894, 11)

In [13]:
from pycaret.classification import *
cls = setup(
    # random_state
    session_id=2020,
    # data yang digunakan untuk membangun model, split data validation, dan target
    data=dfTrain,
    train_size=0.8,
    target='is_canceled',
    # handling multicollinearity
    remove_multicollinearity=True,
    multicollinearity_threshold=0.8,
    # handling outlier
    remove_outliers=True,
    outliers_method='lof',
    outliers_threshold = 0.05,
    # re-scaling data
    normalize=True,
    normalize_method='robust',
    # handling imbalance label
    fix_imbalance=True,
    fix_imbalance_method="SMOTE"
)

,Description,Value
0,Session id,2020
1,Target,is_canceled
2,Target type,Binary
3,Original data shape,"(62679, 11)"
4,Transformed data shape,"(72568, 31)"
5,Transformed train set shape,"(60032, 31)"
6,Transformed test set shape,"(12536, 31)"
7,Numeric features,5
8,Categorical features,5
9,Rows with missing values,0.4%


In [14]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7964,0.8924,0.7937,0.6962,0.7417,0.5749,0.5784,1.6250
gbc,Gradient Boosting Classifier,0.7911,0.8863,0.7854,0.6901,0.7346,0.5635,0.5668,4.9240
rf,Random Forest Classifier,0.7892,0.8850,0.7958,0.6838,0.7355,0.5620,0.5665,3.9840
et,Extra Trees Classifier,0.7883,0.8800,0.7910,0.6839,0.7335,0.5594,0.5636,5.0800
dt,Decision Tree Classifier,0.7862,0.8716,0.7902,0.6807,0.7313,0.5554,0.5598,0.9740
lr,Logistic Regression,0.7793,0.8638,0.7489,0.6825,0.7142,0.5350,0.5366,15.6550
ridge,Ridge Classifier,0.7765,0.0000,0.7394,0.6811,0.7090,0.5280,0.5293,1.1520
lda,Linear Discriminant Analysis,0.7765,0.8569,0.7395,0.6811,0.7090,0.5281,0.5294,1.4040
ada,Ada Boost Classifier,0.7763,0.8684,0.7361,0.6819,0.7078,0.5270,0.5282,2.2930
knn,K Neighbors Classifier,0.7739,0.8379,0.6830,0.6974,0.6900,0.5122,0.5124,4.0340


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=2020, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)